# Back Testing for AshBot

This notebook will make a demonstration for backtesting a trade setup for a  potential client.
As an example, I am going to use **Nifty 500 scrips** and collect data for *60 days* at *30 min* intervals



### Algorithm:
***************************************************************************
*Actual algorithm has been redacted to preserve IP of the strategy builder.* 
****************************************************************************
This will require total 4 indicators aoart from the regular OHLC data. 
Three of those indicatoes are covered in the library *TA_lib* While I have found a seaprate code for implelemting Supertrend. 

### Assumption:
Only one order will be active at a time for any given scrip. Second order will not be initiated until the first order is closed.


## Fetch data 

Import all necesary libraries and download data rom yahoo finance in pandas dataframe.

**Future update:** 
Enable data fetching from multiple sources as necessary and feasible, with customisable parameters for time period, intervals, etc.

In [1]:
# import required libraries
import csv
# import os
import datetime
import time
import numpy as np
#import settings
#import webbrowser
import requests
import yfinance as yf
import talib as ta
import pandas as pd
from dateutil.parser import parse


def FetchData(scrip):
    global data
    # Get data from yahoo finance testing for 30 mins
    data = yf.download(tickers=scrip, period="60d", interval="30m")
    #data.head()
    #data.tail()

## Calculate and update indicators:


Define function for SuperTrend indicator which is not included in *TA_lib.* and apply all indicators to the above dataframe.

**Future update:**
Define a wrapper fucntion to calculate and populate necessary indicators only, which can be customised at runtime.

In [2]:
# EMA function which is required by ATR function.
def EMA(df, base, target, period, alpha=False):
    """
    Function to compute Exponential Moving Average (EMA)
    
    Args :
        df : Pandas DataFrame which contains ['date', 'open', 'high', 'low', 'close', 'volume'] columns
        base : String indicating the column name from which the EMA needs to be computed from
        target : String indicates the column name to which the computed data needs to be stored
        period : Integer indicates the period of computation in terms of number of candles
        alpha : Boolean if True indicates to use the formula for computing EMA using alpha (default is False)
        
    Returns :
        df : Pandas DataFrame with new column added with name 'target'
    """

    con = pd.concat([df[:period][base].rolling(window=period).mean(), df[period:][base]])
    
    if (alpha == True):
        # (1 - alpha) * previous_val + alpha * current_val where alpha = 1 / period
        df[target] = con.ewm(alpha=1 / period, adjust=False).mean()
    else:
        # ((current_val - previous_val) * coeff) + previous_val where coeff = 2 / (period + 1)
        df[target] = con.ewm(span=period, adjust=False).mean()
    
    df[target].fillna(0, inplace=True)
    return df

#ATR function defined, which is necessary to calculate Super Trend
def ATR(df, period, ohlc=['Open', 'High', 'Low', 'Close']):
    """
    Function to compute Average True Range (ATR)
    
    Args :
        df : Pandas DataFrame which contains ['date', 'open', 'high', 'low', 'close', 'volume'] columns
        period : Integer indicates the period of computation in terms of number of candles
        ohlc: List defining OHLC Column names (default ['Open', 'High', 'Low', 'Close'])
        
    Returns :
        df : Pandas DataFrame with new columns added for 
            True Range (TR)
            ATR (ATR_$period)
    """
    atr = 'ATR_' + str(period)

    # Compute true range only if it is not computed and stored earlier in the df
    if not 'TR' in df.columns:
        df['h-l'] = df[ohlc[1]] - df[ohlc[2]]
        df['h-yc'] = abs(df[ohlc[1]] - df[ohlc[3]].shift())
        df['l-yc'] = abs(df[ohlc[2]] - df[ohlc[3]].shift())
         
        df['TR'] = df[['h-l', 'h-yc', 'l-yc']].max(axis=1)
         
        df.drop(['h-l', 'h-yc', 'l-yc'], inplace=True, axis=1)

    # Compute EMA of true range using ATR formula after ignoring first row
    EMA(df, 'TR', atr, period, alpha=True)
    
    return df

def SuperTrend(df, period, multiplier, ohlc=['Open', 'High', 'Low', 'Close']):
    """
    Function to compute SuperTrend
    
    Args :
        df : Pandas DataFrame which contains ['date', 'open', 'high', 'low', 'close', 'volume'] columns
        period : Integer indicates the period of computation in terms of number of candles
        multiplier : Integer indicates value to multiply the ATR
        ohlc: List defining OHLC Column names (default ['Open', 'High', 'Low', 'Close'])
        
    Returns :
        df : Pandas DataFrame with new columns added for 
            True Range (TR), ATR (ATR_$period)
            SuperTrend (ST_$period_$multiplier)
            SuperTrend Direction (STX_$period_$multiplier)
    """

    ATR(df, period, ohlc=ohlc)
    atr = 'ATR_' + str(period)
    st = 'ST_' + str(period) + '_' + str(multiplier)
    stx = 'STX_' + str(period) + '_' + str(multiplier)
    
    """
    SuperTrend Algorithm :
    
        BASIC UPPERBAND = (HIGH + LOW) / 2 + Multiplier * ATR
        BASIC LOWERBAND = (HIGH + LOW) / 2 - Multiplier * ATR
        
        FINAL UPPERBAND = IF( (Current BASICUPPERBAND < Previous FINAL UPPERBAND) or (Previous Close > Previous FINAL UPPERBAND))
                            THEN (Current BASIC UPPERBAND) ELSE Previous FINALUPPERBAND)
        FINAL LOWERBAND = IF( (Current BASIC LOWERBAND > Previous FINAL LOWERBAND) or (Previous Close < Previous FINAL LOWERBAND)) 
                            THEN (Current BASIC LOWERBAND) ELSE Previous FINAL LOWERBAND)
        
        SUPERTREND = IF((Previous SUPERTREND = Previous FINAL UPPERBAND) and (Current Close <= Current FINAL UPPERBAND)) THEN
                        Current FINAL UPPERBAND
                    ELSE
                        IF((Previous SUPERTREND = Previous FINAL UPPERBAND) and (Current Close > Current FINAL UPPERBAND)) THEN
                            Current FINAL LOWERBAND
                        ELSE
                            IF((Previous SUPERTREND = Previous FINAL LOWERBAND) and (Current Close >= Current FINAL LOWERBAND)) THEN
                                Current FINAL LOWERBAND
                            ELSE
                                IF((Previous SUPERTREND = Previous FINAL LOWERBAND) and (Current Close < Current FINAL LOWERBAND)) THEN
                                    Current FINAL UPPERBAND
    """
    
    # Compute basic upper and lower bands
    df['basic_ub'] = (df[ohlc[1]] + df[ohlc[2]]) / 2 + multiplier * df[atr]
    df['basic_lb'] = (df[ohlc[1]] + df[ohlc[2]]) / 2 - multiplier * df[atr]

    # Compute final upper and lower bands
    df['final_ub'] = 0.00
    df['final_lb'] = 0.00
    for i in range(period, len(df)):
        df['final_ub'].iat[i] = df['basic_ub'].iat[i] if df['basic_ub'].iat[i] < df['final_ub'].iat[i - 1] or df[ohlc[3]].iat[i - 1] > df['final_ub'].iat[i - 1] else df['final_ub'].iat[i - 1]
        df['final_lb'].iat[i] = df['basic_lb'].iat[i] if df['basic_lb'].iat[i] > df['final_lb'].iat[i - 1] or df[ohlc[3]].iat[i - 1] < df['final_lb'].iat[i - 1] else df['final_lb'].iat[i - 1]
       
    # Set the Supertrend value
    df[st] = 0.00
    for i in range(period, len(df)):
        df[st].iat[i] = df['final_ub'].iat[i] if df[st].iat[i - 1] == df['final_ub'].iat[i - 1] and df[ohlc[3]].iat[i] <= df['final_ub'].iat[i] else \
                        df['final_lb'].iat[i] if df[st].iat[i - 1] == df['final_ub'].iat[i - 1] and df[ohlc[3]].iat[i] >  df['final_ub'].iat[i] else \
                        df['final_lb'].iat[i] if df[st].iat[i - 1] == df['final_lb'].iat[i - 1] and df[ohlc[3]].iat[i] >= df['final_lb'].iat[i] else \
                        df['final_ub'].iat[i] if df[st].iat[i - 1] == df['final_lb'].iat[i - 1] and df[ohlc[3]].iat[i] <  df['final_lb'].iat[i] else 0.00 
                 
    # Mark the trend direction up/down
   # df[stx] = np.where((df[st] > 0.00), np.where((df[ohlc[3]] < df[st]), 'down',  'up'), np.NaN)

    # Remove basic and final bands from the columns
    df.drop(['basic_ub', 'basic_lb', 'final_ub', 'final_lb','TR',atr], inplace=True, axis=1)
    
    df.fillna(0, inplace=True)

    return df

# Add all Indicators required
#SuperTrend(20,2.5)
#EMA(50)
#Macd (12,26,9)
#Adx (7) is > 30
#Rsi (7) > 70

def  CalculateIndicators():
    SuperTrend(data, 20, 2.5)
    data['EMA_50'] = ta.EMA(data['Close'],50)
    data['ADX_7'] = ta.ADX(data['High'],data['Low'], data['Close'], timeperiod=7)
    data['RSI_7'] = ta.RSI(data['Close'],7)
    data['MACD'],data['MACD_SIG'],data['MACD_HIST'] = ta.MACD(data['Close'],12,26,9)

    data.tail()

## Execute trades:

Apply the algorithm to completed dataset with indicators and generate an output dataframe with executed trades with following columns:

*Timestamp | SYMBOL | ACTION(Buy/Sell)  | Price* 

Simultaneously, there will be a running log file generated with commentry based on events that happen while scanning through testing data. Timestamps used in the log file will be timestamps of the test data set. 

*Timestamp | Logged event/action.*

**Future Update:** Make libraries of multiple strategies, and explore possibilities to combile multiple strategies with the help of ML

In [3]:

startTrade = datetime.time(10, 00, 00)
stopTrade = datetime.time(14, 30, 00)

def checkWorkingHours(index):
    #dt = parse(str(index))
    #print(dt.time())
    print(str(index.time()))
    if(index.time() >= startTrade and index.time() < stopTrade):
        print("Trading Hours")
        return True
        
    else:
        return False
        
def LogEvent(timestamp, message):
    
    eventLog = str(timestamp)+ ','+ message + '\n'
    #eventLog.append(logEntry)
    logFile = open('AshBotEventLog.csv','a')
    logFile.write(eventLog)
    #eventLog.to_csv('AshBotEventLog.csv', mode='a', header=False, sep = '\t')
    #saveFile.write(data.to_string(header=False))
    logFile.close()
    print("Logging event")
        
def LogTrade(timestamp, scrip, action, price):
    
    tradeLog = str(timestamp) + ',' + scrip + ',' + action + ',' + price + '\n' 
    tradeFile = open('AshBotTradeLog.csv','a')
    tradeFile.write(tradeLog)
    #eventLog.to_csv('AshBotEventLog.csv', mode='a', header=False, sep = '\t')
    #saveFile.write(data.to_string(header=False))
    tradeFile.close()
    print("Logging Trade")
    
def RunAshBot():    
    stCrossedAbove = False
    stCrossedBelow = True
    stStayedBelow = True
    buyNext = False
    openTrade = False
    for index,row in data.iterrows():

        if(openTrade == True):
            if(row['High'] >= targetPrice):
                openTrade = False
                LogEvent(index,"Target Met @"+str(targetPrice))
                exitPrice = targetPrice
                LogTrade(index, scrip, 'SELL', str(exitPrice))
                targetPrice = 0
                stopPrice = 0
                print('Target Met', exitPrice, index)
            elif(row['Low'] <= stopPrice):
                openTrade = False
                LogEvent(index,"Target Met @"+str(stopPrice))
                exitPrice = stopPrice
                LogTrade(index, scrip, 'SELL', str(exitPrice))
                targetPrice = 0
                stopPrice = 0
                print('StopLoss Hit',exitPrice, index)

        if(buyNext == True):
            buyNext = False
            if(checkWorkingHours(index) == True):
                openTrade = True
                buyPrice = row['Open']
                LogEvent(index,"Buy Executed @"+str(buyPrice))
                LogTrade(index, scrip, 'BUY', str(buyPrice))
                print('Buy Price:', buyPrice, index)
                targetPrice = buyPrice + (buyPrice*2/100)
                stopPrice = buyPrice - (buyPrice*1/100)
            else:
                LogEvent(index,"Triggered outside Trading hours")


#***************************************************************************
#*Actual algorithm has been redacted to preserve IP of the strategy builder.* 
#****************************************************************************
                                if(openTrade == False):
                                    buyNext = True
                                else:
                                    LogEvent(index,"Trade already open")

#***************************************************************************
#*Actual algorithm has been redacted to preserve IP of the strategy builder.* 
#****************************************************************************
        #print(row['Close'])



## Iterate through all Nifty 500 scrips

Read nifty 500 symbols from CSV and iterate through all data to generate test result for all data.

**Future Update:** Create customizable groups on which the stratagies can be run.

In [4]:

global scrip
with open("ind_nifty500list.csv", 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for line in csv_reader:
        print(line['Company Name'])
        scrip = str(line['Symbol'])
        tickerSymb = str(line['Symbol'])+'.NS'
        FetchData(tickerSymb)
        #print(data)
        CalculateIndicators()
        #print(data)
        #data.tail()
        RunAshBot()

3M India Ltd.
[*********************100%***********************]  1 of 1 completed
Logging event
Logging event
Previous candle ST crossed below 2020-08-07 10:00:00+05:30
Logging event
 Closed Higher 20863.94921875
Logging event
Logging event
Previous candle ST crossed below 2020-08-10 14:30:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-13 09:30:00+05:30
Logging event
 Closed Higher 21990.0
Logging event
  Closed Above 50 EMA
Logging event
   MACD above Signal
Logging event
     RSI_7 > 70
Logging event
    ADX 7 > 30
10:00:00
Trading Hours
Logging event
Logging Trade
Buy Price: 21999.94921875 2020-08-13 10:00:00+05:30
Logging event
Logging Trade
StopLoss Hit 21779.9497265625 2020-08-13 12:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-24 10:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-25 09:30:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-27 09:30:00+05:

Logging Trade
Target Met 1436.0580249023437 2020-08-18 15:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-20 10:00:00+05:30
Logging event
 Closed Higher 1438.0
Logging event
  Closed Above 50 EMA
Logging event
Logging event
Previous candle ST crossed below 2020-08-21 09:30:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-21 11:00:00+05:30
Logging event
 Closed Higher 1425.8499755859375
Logging event
  Closed Above 50 EMA
Logging event
Logging event
Previous candle ST crossed below 2020-08-24 09:30:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-28 09:30:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-09-01 15:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-09-07 13:00:00+05:30
Logging event
 Closed Higher 1334.300048828125
Logging event
Logging event
Previous candle ST crossed below 2020-09-08 14:30:00+05:30
Logging event
Logging event
Pre

[*********************100%***********************]  1 of 1 completed
Logging event
Logging event
Previous candle ST crossed below 2020-08-05 10:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-12 14:30:00+05:30
Logging event
 Closed Higher 726.3499755859375
Logging event
  Closed Above 50 EMA
Logging event
   MACD above Signal
Logging event
     RSI_7 > 70
Logging event
Logging event
Previous candle ST crossed below 2020-08-13 13:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-14 10:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-14 11:30:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-19 09:00:00+05:30
Logging event
 Closed Higher 737.5
Logging event
  Closed Above 50 EMA
Logging event
   MACD above Signal
Logging event
     RSI_7 > 70
Logging event
    ADX 7 > 30
09:30:00
Logging event
Logging event
Logging event
Previous candle ST crossed below 2020-08-20 1

[*********************100%***********************]  1 of 1 completed
Logging event
Logging event
Previous candle ST crossed below 2020-08-05 09:00:00+05:30
Logging event
 Closed Higher 16672.099609375
Logging event
Logging event
Previous candle ST crossed below 2020-08-05 15:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-10 09:30:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-13 11:30:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-17 11:30:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-24 09:30:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-27 09:30:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-28 10:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-31 10:00:00+05:30
Logging event
 Closed Higher 16759.80078125
Logging event
  Closed Above 50 EMA
Adani Enterprises Ltd.


[*********************100%***********************]  1 of 1 completed
Logging event
Logging event
Previous candle ST crossed below 2020-08-05 10:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-12 10:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-17 09:30:00+05:30
Logging event
 Closed Higher 355.29998779296875
Logging event
  Closed Above 50 EMA
Logging event
   MACD above Signal
Logging event
     RSI_7 > 70
Logging event
    ADX 7 > 30
10:00:00
Trading Hours
Logging event
Logging Trade
Buy Price: 355.1499938964844 2020-08-17 10:00:00+05:30
Logging event
Logging Trade
Target Met 362.25299377441405 2020-08-19 09:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-19 15:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-21 09:30:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-26 09:30:00+05:30
Logging event
 Closed Higher 364.5
Loggin

...

[*********************100%***********************]  1 of 1 completed
Logging event
Logging event
Previous candle ST crossed below 2020-08-03 15:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-04 10:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-19 11:00:00+05:30
Logging event
 Closed Higher 3440.0
Logging event
  Closed Above 50 EMA
Logging event
   MACD above Signal
Logging event
     RSI_7 > 70
Logging event
    ADX 7 > 30
11:30:00
Trading Hours
Logging event
Logging Trade
Buy Price: 3440.0 2020-08-19 11:30:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-21 14:30:00+05:30
Logging event
Logging Trade
Target Met 3508.8 2020-08-24 10:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-24 11:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-25 09:30:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-27 09:3

[*********************100%***********************]  1 of 1 completed
Logging event
Logging event
Previous candle ST crossed below 2020-08-04 12:30:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-07 11:30:00+05:30
Logging event
 Closed Higher 233.14999389648438
Logging event
  Closed Above 50 EMA
Logging event
   MACD above Signal
Logging event
     RSI_7 > 70
Logging event
    ADX 7 > 30
12:00:00
Trading Hours
Logging event
Logging Trade
Buy Price: 233.14999389648438 2020-08-07 12:00:00+05:30
Logging event
Logging Trade
StopLoss Hit 230.81849395751954 2020-08-07 12:30:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-12 12:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-17 15:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-20 10:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-26 10:30:00+05:30
Logging event
Logging event
Previous c

[*********************100%***********************]  1 of 1 completed
Logging event
Logging event
Previous candle ST crossed below 2020-08-04 09:30:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-05 10:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-06 10:00:00+05:30
WABCO India Ltd.
[*********************100%***********************]  1 of 1 completed
Logging event
Logging event
Previous candle ST crossed below 2020-08-04 12:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-05 10:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-13 11:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-13 12:30:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-21 14:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-25 10:00:00+05:30
Logging event
 Closed Higher 6985.0498046875
Logging event
Log

[*********************100%***********************]  1 of 1 completed
Logging event
Logging event
Previous candle ST crossed below 2020-08-03 15:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-05 11:00:00+05:30
Logging event
 Closed Higher 329.1000061035156
Logging event
Logging event
Previous candle ST crossed below 2020-08-10 10:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-14 10:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-21 10:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-24 09:30:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-27 10:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-27 14:30:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-09-03 12:30:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-09-04 10:30:00+05:30
Logging event


[*********************100%***********************]  1 of 1 completed
Logging event
Logging event
Previous candle ST crossed below 2020-08-13 09:30:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-17 14:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-20 11:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-20 15:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-21 10:00:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-21 13:30:00+05:30
Logging event
Logging event
Previous candle ST crossed below 2020-08-24 10:00:00+05:30
Logging event
 Closed Higher 1643.449951171875
Logging event
  Closed Above 50 EMA
Logging event
   MACD above Signal
Logging event
     RSI_7 > 70
Logging event
    ADX 7 > 30
10:30:00
Trading Hours
Logging event
Logging Trade
Buy Price: 1646.050048828125 2020-08-24 10:30:00+05:30
Logging event
Logging Trade
StopLoss Hi

## Run analytics on the trade output file generated

Generate analytics report that includes:

1. Total trades executed
2. total winning trades
3. Total losing trades
4. Average winning trade
5. Average losing trade
6. Hit Ratio
7. Edge Ratio

In [ ]:
#TBD